In [2]:
import os
import re
import pandas as pd
from glob import glob

In [4]:
parent_dir = "/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences"
bin_range = "125_155"
intersect_folder = "geneIntersect"
# or 
# intersect_folder = "nuclIntersect"

In [6]:
gene_intersect_dir = os.path.join(bin_dir, "bedtool_out")

In [5]:
gene_intersect_dir


'/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/bedtool_out'

In [35]:
def processing_bins(bin_dir, intersect_folder):
    # Collecting bed files from the current bins directory
    intersect_dir = os.path.join(bin_dir, "bedtools_out", intersect_folder)
    bed_files = glob(os.path.join(intersect_dir, "*_genesIntersect.bed"))

    df_list = []

    # TODO: remove this item
    Anumber_lengths = len(bed_files)
    print(f"This is the number of .bed files in the folder: {Anumber_lengths}")
    for bedfile in bed_files:

        # TODO: Remove the prints and remove the first sample_id
        #testing
        sample_id = os.path.basename(bedfile)#.replace("_geneIntersect.bed","")
        print(f"This is the befile: {bedfile}")
        print(f"This is the basename without replacement: {sample_id}")
        sample_id = os.path.basename(bedfile).replace("_genesIntersect.bed","")
        print(f"This is with the replacement: {sample_id}")
        print(" ")
        
   

In [36]:
bin_dir = os.path.join(parent_dir, bin_range)
processing_bins(bin_dir, intersect_folder)

This is the number of .bed files in the folder: 16
This is the befile: /Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/125_155/bedtools_out/geneIntersect/A14902_125_155_genesIntersect.bed
This is the basename without replacement: A14902_125_155_genesIntersect.bed
This is with the replacement: A14902_125_155
 
This is the befile: /Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/125_155/bedtools_out/geneIntersect/A14899_125_155_genesIntersect.bed
This is the basename without replacement: A14899_125_155_genesIntersect.bed
This is with the replacement: A14899_125_155
 
This is the befile: /Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/125_155/bedtools_out/geneIntersect/A14895_125_155_genesIntersect.bed
This is the basename without replacement: A14895_125_155_genesIntersect.bed
This is with the replacement: A14895_125_155
 
This is the befile: /Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/125_155/bedtools_out/gene

In [40]:
bin_range
parent_dir
intersect_folder

'geneIntersect'

In [6]:
test_file = os.path.join(parent_dir, bin_range,"bedtools_out", intersect_folder, "A14891_125_155_genesIntersect.bed")
print(test_file)

/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/125_155/bedtools_out/geneIntersect/A14891_125_155_genesIntersect.bed


In [7]:
df_test = pd.read_csv(
    test_file,
    sep='\t',
    header=None,
    names=["chr","source","feature","start","end","score","strand","phase","attributes","count"]
    )

# subsetting for testing
df_sampled = df_test.sample(n=20, random_state=42)

In [49]:
df_test["count"].describe()

count    21542.000000
mean       112.787578
std        219.939016
min          0.000000
25%         14.000000
50%         44.000000
75%        120.000000
max       4291.000000
Name: count, dtype: float64

In [30]:
df_test["gene_id"] = df_test["attributes"].apply(
    lambda x: re.search(r"gene:([^;]+)", x).group(1) if re.search(r"gene:([^;]+)", x) else None
)
df_test["gene_name"] = df_test["attributes"].apply(
    lambda x: re.search(r"Name=([^;]+)", x).group(1) if re.search(r"Name=([^;]+)", x) else None
)

In [31]:
df_test.head()

,chr,source,feature,start,end,score,strand,phase,attributes,count,gene_id,gene_name
0,1,ensembl_havana,gene,65419,71585,.,+,.,ID=gene:ENSG00000186092;Name=OR4F5;biotype=pro...,14,ENSG00000186092,OR4F5
1,1,ensembl_havana,gene,450740,451678,.,-,.,ID=gene:ENSG00000284733;Name=OR4F29;biotype=pr...,2,ENSG00000284733,OR4F29
2,1,ensembl_havana,gene,685716,686654,.,-,.,ID=gene:ENSG00000284662;Name=OR4F16;biotype=pr...,0,ENSG00000284662,OR4F16
3,1,ensembl_havana,gene,923923,944575,.,+,.,ID=gene:ENSG00000187634;Name=SAMD11;biotype=pr...,40,ENSG00000187634,SAMD11
4,1,ensembl_havana,gene,944203,959309,.,-,.,ID=gene:ENSG00000188976;Name=NOC2L;biotype=pro...,28,ENSG00000188976,NOC2L


In [32]:
df_group_counts = df_test.groupby(["gene_id", "gene_name"]).size().reset_index(name="count_occurrences")

# Filter only those with 2 or more occurrences
df_duplicates = df_group_counts[df_group_counts["count_occurrences"] >= 2]

# Display the duplicate groups
print(df_duplicates)


Empty DataFrame
Columns: [gene_id, gene_name, count_occurrences]
Index: []


In [33]:
df_group_counts = df_test.groupby(["gene_id", "gene_name"]).size().reset_index(name="count_occurrences")

# Display the first few rows of the grouped counts
print(df_group_counts.head())


           gene_id gene_name  count_occurrences
0  ENSG00000000003    TSPAN6                  1
1  ENSG00000000005      TNMD                  1
2  ENSG00000000419      DPM1                  1
3  ENSG00000000457     SCYL3                  1
4  ENSG00000000460     FIRRM                  1


In [34]:
print(df_test["gene_id"].value_counts().head(20))  # Check the top 20 most frequent gene IDs


gene_id
ENSG00000186092    1
ENSG00000068745    1
ENSG00000178035    1
ENSG00000178057    1
ENSG00000178149    1
ENSG00000178252    1
ENSG00000178467    1
ENSG00000177479    1
ENSG00000178537    1
ENSG00000114302    1
ENSG00000213672    1
ENSG00000162267    1
ENSG00000008300    1
ENSG00000225697    1
ENSG00000183396    1
ENSG00000010256    1
ENSG00000114270    1
ENSG00000145040    1
ENSG00000114268    1
ENSG00000164054    1
Name: count, dtype: int64


In [55]:
import os
import re
import pandas as pd
from glob import glob

def process_bin_gene_counts(bin_dir, intersect_folder):
    gene_intersect_dir = os.path.join(bin_dir, "bedtools_out", intersect_folder)
    
    bed_files = glob(os.path.join(gene_intersect_dir, "*_genesIntersect.bed"))
    df_list = []
    
    for bedfile in bed_files:
        # e.g. "A14891_125_155_genesIntersect.bed"
        sample_id = os.path.basename(bedfile).replace("_genesIntersect.bed","")  # "A14891_125_155"
        
        # Read the BED file
        # By default, pandas will treat # lines as comments if you have them. 
        # Adjust header=None if no header line in the file.
        print(f"Reading {sample_id}")
        df = pd.read_csv(
            bedfile,
            sep='\t',
            header=None,
            names=["chr","source","feature","start","end","score","strand","phase","attributes","count"]
        )
        
        # Parse gene_id and gene_name from the 'attributes' column
        # Look for strings like "ID=gene:ENSG00000186092;Name=OR4F5;..."
        # We'll capture "ENSG00000186092" and "OR4F5" with regex
        df["gene_id"] = df["attributes"].apply(
            lambda x: re.search(r"gene:([^;]+)", x).group(1) if re.search(r"gene:([^;]+)", x) else None
        )
        df["gene_name"] = df["attributes"].apply(
            lambda x: re.search(r"Name=([^;]+)", x).group(1) if re.search(r"Name=([^;]+)", x) else None
        )
        
        # Aggregate counts by gene
        df_sum = df.groupby(["gene_id","gene_name"], as_index=False)["count"].sum()
        
        # Attach sample info
        df_sum["sample"] = sample_id
        df_list.append(df_sum)

    print("Finished reading files")
    print("Combining.....")
    # Combine into one long df
    combined = pd.concat(df_list, ignore_index=True)
    print("Finished combining")
    # Pivot to wide format
    wide_counts = combined.pivot_table(
        index=["gene_id","gene_name"], 
        columns="sample", 
        values="count",
        fill_value=0
    ).reset_index()
    
    return wide_counts

In [56]:
# Example usage
# bin_dir = "/Users/.../ctDNA_11042024/data/human_binned_sequences/125_155"
# counts_125_155 = process_bin_gene_counts(bin_dir)
# counts_125_155.to_csv("125_155_gene_counts.tsv", sep="\t", index=False)

# If you want to do the loop for each bin
bins = ["125_155","150_180","170_220","220_345","40_150"]
parent = "/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences"
output_dir = "/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/ctMatrices/"
#Which Counts?

In [57]:
intersect_type = 'gene'

if intersect_type == 'gene':
    intersect_folder = "geneIntersect"
    sub_folder = "geneCounts"
    out_name = "_gene_counts.tsv"
else:
    intersect_folder = "nuclIntersect"
    sub_folder = "nuclCounts"
    out_name = "_nucleosome_counts.tsv"

print(intersect_folder)

geneIntersect


In [58]:

for b in bins:
    bin_path = os.path.join(parent, b)
    mat = process_bin_gene_counts(bin_path, intersect_folder)
    outname = os.path.join(output_dir, sub_folder, f"{b}{out_name}")
    mat.to_csv(outname, sep="\t", index=False)

Reading A14902_125_155
Reading A14899_125_155
Reading A14895_125_155
Reading A14892_125_155
Reading A14905_125_155
Reading A14893_125_155
Reading A14904_125_155
Reading A14903_125_155
Reading A14898_125_155
Reading A14894_125_155
Reading A14906_125_155
Reading A14891_125_155
Reading A14896_125_155
Reading A14901_125_155
Reading A14897_125_155
Reading A14900_125_155
Finished reading files
Combining.....
Finished combining
Reading A14892_150_180
Reading A14905_150_180
Reading A14902_150_180


KeyboardInterrupt: 